In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py
import seaborn as sns
from pathlib import Path
from nptdms import TdmsFile #Process ramping file
%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
/uscms_data/d3/dhoang/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [5]:
!ls 15T_data/time_axis

Ramp_10_time.npy   Ramp_26-_time.npy  Ramp_40_time.npy	 Ramp_57_time.npy
Ramp_11_time.npy   Ramp_27-_time.npy  Ramp_41-_time.npy  Ramp_5_time.npy
Ramp_12_time.npy   Ramp_28-_time.npy  Ramp_42_time.npy	 Ramp_6_time.npy
Ramp_14-_time.npy  Ramp_29-_time.npy  Ramp_43_time.npy	 Ramp_76_time.npy
Ramp_15-_time.npy  Ramp_2_time.npy    Ramp_44-_time.npy  Ramp_77-_time.npy
Ramp_16_time.npy   Ramp_30-_time.npy  Ramp_45-_time.npy  Ramp_78-_time.npy
Ramp_17-_time.npy  Ramp_31-_time.npy  Ramp_46-_time.npy  Ramp_79-_time.npy
Ramp_18-_time.npy  Ramp_32-_time.npy  Ramp_47-_time.npy  Ramp_7_time.npy
Ramp_19_time.npy   Ramp_33-_time.npy  Ramp_48-_time.npy  Ramp_80-_time.npy
Ramp_1_time.npy    Ramp_34-_time.npy  Ramp_49-_time.npy  Ramp_81-_time.npy
Ramp_20-_time.npy  Ramp_35-_time.npy  Ramp_4_time.npy	 Ramp_82-_time.npy
Ramp_21-_time.npy  Ramp_36-_time.npy  Ramp_50-_time.npy  Ramp_84-_time.npy
Ramp_22-_time.npy  Ramp_37-_time.npy  Ramp_53-_time.npy  Ramp_85-_time.npy
Ramp_23-_time.npy  Ramp_38-_time.npy  

In [14]:
def calculate_mat_time(ramp_num, dir_path):
    """
    Take a ramp's mat (MATHLAB produced) file and calculate its time axis from the current data 
    (set maximum current point at 0 and multiply by sampling rate). 
    
    Example usage: calculate_mat_time("Ramp_1", "15T_data")
    
    
    Args:
        - ramp_num (str): ramp number
        - dir_path (str): path to directory containing all the ramp's data
        
    Returns: None, save the calculated time axis to a npy file in dir_path + "/time_axis/"
    """
    
    needed_file = None
    
    #Scan the directory for correct ramp file
    for filename in os.listdir(dir_path):
        if filename.startswith(ramp_num + '-'):
            needed_file = dir_path + "/" + filename
            
    #Load the file
    f = h5py.File(needed_file,'r')
    data = np.array(f['chanvals'], dtype = 'float16')
    current_data = data[3, :]
    current_data = np.nan_to_num(current_data)
    
    #Add time axis relative to the time when quench happens
    time_range = np.asarray(range(current_data.shape[0]))
    
    #Calculate its time axis, center around the max value (quench happens at 0 time)
    max_index = np.argmax(current_data)
    time_range -= max_index
    time_range = time_range.astype('float32')
    
    #Multiply by datarate
    time_range = np.multiply(time_range, 1e-5, out=time_range, casting="unsafe")
    
    #Save the time axis
    np.save(dir_path + "/time_axis/" + ramp_num + "_time", time_range)
    
    %reset -f in
    
def auto_time_cal(dir_path):
    """
    Scan a directory and automatically calculate the time axis for ramps that don't have a time_axis
    """
    
    available_ramps = ['Ramp_' + str(x) for x in list(range(1,90))]
    
    
    for ramp_num in available_ramps:
        print("Checking {} ...".format(ramp_num))
        #Scan every ramp in the directory
        for filename in os.listdir(dir_path):
            if filename.startswith(ramp_num + '-'):
                #Check if the time axis for this ramp exists
                time_file = Path(dir_path + "/time_axis/" + ramp_num + "_time.npy")
                
                if not time_file.exists():
                    #If not, calculate the time axis of this ramp
                    print("Calculating time for", ramp_num)
                    calculate_mat_time(ramp_num, dir_path)
                    
                    
def rename_time_file():
    
    dir_path = "./15T_data/time_axis/"
    
    available_ramps = ['Ramp_' + str(x) for x in list(range(1,90))]
    
    
    for ramp_num in available_ramps:
        print("Checking {} ...".format(ramp_num))
        #Scan every ramp in the directory
        for filename in os.listdir(dir_path):
            if filename.startswith(ramp_num + '-'):
                print("renaming {} to {}".format(dir_path + filename, dir_path + ramp_num + "_time.npy"))
                os.rename(dir_path + filename, dir_path + ramp_num + "_time.npy")

In [16]:
!ls 15T_data/time_axis

Ramp_10_time.npy  Ramp_26_time.npy  Ramp_40_time.npy  Ramp_57_time.npy
Ramp_11_time.npy  Ramp_27_time.npy  Ramp_41_time.npy  Ramp_5_time.npy
Ramp_12_time.npy  Ramp_28_time.npy  Ramp_42_time.npy  Ramp_6_time.npy
Ramp_14_time.npy  Ramp_29_time.npy  Ramp_43_time.npy  Ramp_76_time.npy
Ramp_15_time.npy  Ramp_2_time.npy   Ramp_44_time.npy  Ramp_77_time.npy
Ramp_16_time.npy  Ramp_30_time.npy  Ramp_45_time.npy  Ramp_78_time.npy
Ramp_17_time.npy  Ramp_31_time.npy  Ramp_46_time.npy  Ramp_79_time.npy
Ramp_18_time.npy  Ramp_32_time.npy  Ramp_47_time.npy  Ramp_7_time.npy
Ramp_19_time.npy  Ramp_33_time.npy  Ramp_48_time.npy  Ramp_80_time.npy
Ramp_1_time.npy   Ramp_34_time.npy  Ramp_49_time.npy  Ramp_81_time.npy
Ramp_20_time.npy  Ramp_35_time.npy  Ramp_4_time.npy   Ramp_82_time.npy
Ramp_21_time.npy  Ramp_36_time.npy  Ramp_50_time.npy  Ramp_84_time.npy
Ramp_22_time.npy  Ramp_37_time.npy  Ramp_53_time.npy  Ramp_85_time.npy
Ramp_23_time.npy  Ramp_38_time.npy  Ramp_54_time.npy  Ramp_86_time.npy
Ramp_24_t